
Originally created by Chelsey McGowan-Yallop, SAMS-UHI (sa06cm@sams.ac.uk)

Modified by Muriel Dunn for AFKABAN then AZKABAN

This script uses model-predicted TS(f) spectra to train a machine learning
classifier, performs nested cross-validation, applies the classifier to
measured TS(f) spectra and outputs results files.

To use a different classifier, see the list of supported classifiers at:
https://github.com/hyperopt/hyperopt-sklearn and set as clf.

Sometimes the initial hyperparameter configuration selected by HyperOpt in each
split in the outer loop will be unsuccessful and all trials will fail. The
retry decorator forces it to try again until retry_limit is reached.

OUTPUT FILES:
    _NestedCV.pkl contains results of nested cross-validation procedure
    _Predictions.pkl contains measured TS(f) spectra with predicted labels
    _BestParams.pkl contains the optimal hyperparameters for the model
"""

In [1]:
import time
import os.path
import numpy as np
import pandas as pd
import pickle
import scipy
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, confusion_matrix
#from sklearn.neighbors import KNeighborsClassifier
import hyperopt
from hyperopt import tpe
from hpsklearn import HyperoptEstimator, k_neighbors_classifier, svc, lightgbm_classification, gaussian_nb, any_preprocessing,min_max_scaler, normalizer
import lightgbm
from datetime import timedelta, date
from tenacity import retry, stop_after_attempt
import AZKABANML
import sys, errno  


# Read the dataframes

In [2]:
# -- USER-DETERMINED PARAMETERS -----------------------------------------------
path_df = 'C:/Users/mbd/OneDrive - Akvaplan-niva AS/PhD-APN/ChaptersandExperiments/AZKABAN-light/ZoopMix_paper/'

# Mesocosm measurements
df_SED = pd.read_pickle(path_df+'df_SED.pkl')
df_track = pd.read_pickle(path_df+'df_track.pkl')
df_trackavg = pd.read_pickle(path_df+'df_trackavg.pkl')

# Labelled training set (scattering model results with smoothed shape file)
df_sim_smooth = pd.read_pickle(path_df+'df_sim_smooth.pkl')
df_sim_smooth_a = pd.read_pickle(path_df+'df_sim_smooth_antarctica.pkl')

### Partition modeled target spectra

In [3]:
### select bandwith 200
df_sim_200 = df_sim_smooth.loc[:,185.0:255.0].copy()
species_data = df_sim_smooth.loc[:,'spec']
df_sim_200.loc[:,'Species'] = species_data

# 70 kHz
df_sim_70 = df_sim_smooth.loc[:,45.0:90.0].copy()
df_sim_70.loc[:,'Species'] = species_data

# 120 kHz
df_sim_120 = df_sim_smooth.loc[:,90.0:170.0].copy()
df_sim_120.loc[:,'Species'] = species_data

# 333 kHz
df_sim_333 = df_sim_smooth.loc[:,283.0:383.0].copy()
df_sim_333.loc[:,'Species'] = species_data

# full kHz
df_sim_full = df_sim_smooth.loc[:,45.0:383.0].copy()
df_sim_full.loc[:,'Species'] = species_data

# Run different iterations


## Preprocessing
Any! Give better results (+~0.2 on mean F1 score), either scale or min,max scaler or PCA.

In [4]:
# -- USER-DETERMINED PARAMETERS -----------------------------------------------
path = 'F:/Nya_AZKABAN/'
classifypath = 'F:/Nya-AZKABAN/Classify/'
d1 = '14-06-2023'#date.today().strftime("%d-%m-%Y")

# NESTED CROSS-VALIDATION
#preprocessing = [min_max_scaler(unique_id)] # List of sklearn pre-processing modules
ex_preprocessing = [] # As above, see help(HyperoptEstimator) for info
n_splits = 10 # Value of k for k-fold cross-validation in outer loop
n_folds = 10 # Value of k for k-fold cross-validation in inner loop
max_evals = 10 # No. of HyperOpt trials
timeout = 300 # HyperOpt trial timeout (seconds)
n_jobs = -1 # No. of jobs to run in parallel; -1 uses all processors
retry_limit = 3 # No. of times to retry before failing

### kNN

In [5]:
# CLASSIFIER kNN 200
unique_id = 'kNN_'+d1+'_smooth' # Unique ID for output file paths
clf = k_neighbors_classifier(unique_id)  # Classifier
AZKABANML.main_classify_all(df_sim_200, df_SED, df_track, df_trackavg, clf, unique_id, classifypath)

Resuming cross-validation from fold 11
Retraining model on full dataset
100%|████████████████████████████████████████████████| 50/50 [00:07<00:00,  7.59s/trial, best loss: 0.3267385467222379]
Classifying new data


In [6]:
# CLASSIFIER kNN normalizer
unique_id = 'kNN_'+d1+'_smooth_norm' # Unique ID for output file paths
clf = k_neighbors_classifier(unique_id)  # Classifier
AZKABANML.main_classify_all(df_sim_200, df_SED, df_track, df_trackavg, clf, unique_id, classifypath, preprocessing=[normalizer(unique_id, norm='l2')])

Resuming cross-validation from fold 11
Retraining model on full dataset
100%|████████████████████████████████████████████████| 50/50 [00:06<00:00,  6.21s/trial, best loss: 0.3001387490032339]
Classifying new data


In [7]:
unique_id = 'lightGBM_'+d1+'_smooth_norm' #que ID for output file paths
clf = lightgbm_classification(unique_id)  # Classifier
AZKABANML.main_classify_all(df_sim_200, df_SED, df_track, df_trackavg, clf, unique_id, classifypath, preprocessing=[normalizer(unique_id, norm='l2')])

Resuming cross-validation from fold 9
Beginning fold 9 of 10
100%|███████████████████████████████████████████████████████████████████| 1/1 [05:03<00:00, 303.96s/trial, best loss=?]
Resuming cross-validation from fold 9
Beginning fold 9 of 10
100%|███████████████████████████████████████████████| 50/50 [03:51<00:00, 231.75s/trial, best loss: 0.2915634307272623]
Beginning fold 10 of 10
100%|███████████████████████████████████████████████████████████████████| 1/1 [05:04<00:00, 304.54s/trial, best loss=?]
Resuming cross-validation from fold 10
Beginning fold 10 of 10
100%|███████████████████████████████████████████████| 14/14 [00:41<00:00, 41.13s/trial, best loss: 0.29349574685973434]


100%|███████████████████████████████████████████████| 50/50 [01:02<00:00, 62.86s/trial, best loss: 0.28626383807344247]
Retraining model on full dataset
100%|██████████████████████████████████████████████| 32/32 [05:04<00:00, 304.63s/trial, best loss: 0.28692327712021526]


100%|██████████████████████████████████████████████| 50/50 [02:40<00:00, 160.20s/trial, best loss: 0.28692327712021526]
Classifying new data


In [5]:
unique_id = 'SVC_'+d1+'_smooth_norm' #que ID for output file paths
clf = svc(unique_id)  # Classifier
AZKABANML.main_classify_all(df_sim_200, df_SED, df_track, df_trackavg, clf, unique_id, classifypath, preprocessing=[normalizer(unique_id, norm='l2')])

Resuming cross-validation from fold 10
Beginning fold 10 of 10
100%|██████████████████████████████████████████████| 50/50 [05:04<00:00, 304.40s/trial, best loss: 0.40975709057255716]
Retraining model on full dataset
100%|████████████████████████████████████████████████| 17/17 [00:16<00:00, 16.67s/trial, best loss: 0.4392329499378651]


100%|████████████████████████████████████████████████| 50/50 [00:17<00:00, 17.43s/trial, best loss: 0.4016067004086973]
Classifying new data


## Bandwidth runs

In [6]:
# CLASSIFIER kNN with bandwidth chuncks
unique_id = 'kNN_'+ d1 +'_70' # Unique ID for output file paths
clf = k_neighbors_classifier(unique_id)  # Classifier
AZKABANML.main_classify(df_sim_70, clf, unique_id, classifypath, preprocessing=[normalizer(unique_id, norm='l2')])

Beginning fold 1 of 10
100%|████████████████████████████████████████████████| 10/10 [00:05<00:00,  5.06s/trial, best loss: 0.2377290753628316]
Beginning fold 2 of 10
100%|████████████████████████████████████████████████| 10/10 [00:06<00:00,  6.63s/trial, best loss: 0.2293621202146663]
Beginning fold 3 of 10
100%|███████████████████████████████████████████████| 10/10 [00:07<00:00,  7.28s/trial, best loss: 0.23264729523759053]
Beginning fold 4 of 10
100%|████████████████████████████████████████████████| 10/10 [00:11<00:00, 11.07s/trial, best loss: 0.2278250134685913]
Beginning fold 5 of 10
100%|███████████████████████████████████████████████| 10/10 [00:04<00:00,  4.63s/trial, best loss: 0.22332941252337124]
Beginning fold 6 of 10
100%|███████████████████████████████████████████████| 10/10 [00:04<00:00,  4.66s/trial, best loss: 0.23198822525071006]
Beginning fold 7 of 10
100%|█████████████████████████████████████████████████| 6/6 [00:04<00:00,  4.85s/trial, best loss: 0.23758402078162755]

100%|████████████████████████████████████████████████| 10/10 [00:04<00:00,  4.62s/trial, best loss: 0.2319861930152235]
Beginning fold 8 of 10
100%|███████████████████████████████████████████████| 10/10 [00:04<00:00,  4.67s/trial, best loss: 0.22126578111321615]
Beginning fold 9 of 10
100%|████████████████████████████████████████████████| 10/10 [00:07<00:00,  7.81s/trial, best loss: 0.2314355739274111]
Beginning fold 10 of 10
100%|███████████████████████████████████████████████| 10/10 [00:06<00:00,  6.94s/trial, best loss: 0.22978473955670176]
Retraining model on full dataset
100%|███████████████████████████████████████████████| 10/10 [00:13<00:00, 13.82s/trial, best loss: 0.22365225462770888]


In [7]:
# CLASSIFIER kNN with bandwidth chuncks
unique_id = 'kNN_'+ d1 +'_120' # Unique ID for output file paths
clf = k_neighbors_classifier(unique_id)  # Classifier
AZKABANML.main_classify(df_sim_120, clf, unique_id, classifypath, preprocessing=[normalizer(unique_id, norm='l2')])

Beginning fold 1 of 10
100%|███████████████████████████████████████████████| 10/10 [00:06<00:00,  6.35s/trial, best loss: 0.19581594377042788]
Beginning fold 2 of 10
100%|███████████████████████████████████████████████| 10/10 [00:05<00:00,  5.94s/trial, best loss: 0.19740015222718943]
Beginning fold 3 of 10
100%|████████████████████████████████████████████████| 10/10 [00:07<00:00,  7.01s/trial, best loss: 0.1971977416220807]
Beginning fold 4 of 10
100%|███████████████████████████████████████████████| 10/10 [00:14<00:00, 14.85s/trial, best loss: 0.18489781014856133]
Beginning fold 5 of 10
100%|█████████████████████████████████████████████████| 10/10 [00:06<00:00,  6.88s/trial, best loss: 0.202602550086604]
Beginning fold 6 of 10
100%|█████████████████████████████████████████████████| 10/10 [00:06<00:00,  6.32s/trial, best loss: 0.200862122728661]
Beginning fold 7 of 10
100%|█████████████████████████████████████████████████| 6/6 [00:06<00:00,  6.27s/trial, best loss: 0.19817463854754014]

100%|███████████████████████████████████████████████| 10/10 [00:04<00:00,  4.95s/trial, best loss: 0.19817463854754014]
Beginning fold 8 of 10
100%|███████████████████████████████████████████████| 10/10 [00:05<00:00,  5.07s/trial, best loss: 0.18750197963069526]
Beginning fold 9 of 10
100%|███████████████████████████████████████████████| 10/10 [00:06<00:00,  6.33s/trial, best loss: 0.19721039186523925]
Beginning fold 10 of 10
100%|███████████████████████████████████████████████| 10/10 [00:04<00:00,  4.96s/trial, best loss: 0.19845335554138632]
Retraining model on full dataset
100%|███████████████████████████████████████████████| 10/10 [00:19<00:00, 19.46s/trial, best loss: 0.19427528303818953]


In [9]:
# CLASSIFIER kNN with bandwidth chuncks
unique_id = 'kNN_'+ d1 +'_383' # Unique ID for output file paths
clf = k_neighbors_classifier(unique_id)  # Classifier
AZKABANML.main_classify(df_sim_333, clf, unique_id, classifypath, preprocessing=[normalizer(unique_id, norm='l2')])

Beginning fold 1 of 10
100%|████████████████████████████████████████████████| 10/10 [00:06<00:00,  6.32s/trial, best loss: 0.3673612640079319]
Beginning fold 2 of 10
100%|███████████████████████████████████████████████| 10/10 [00:07<00:00,  7.16s/trial, best loss: 0.36415498347131003]
Beginning fold 3 of 10
100%|████████████████████████████████████████████████| 10/10 [00:06<00:00,  6.21s/trial, best loss: 0.3631041976384626]
Beginning fold 4 of 10
100%|████████████████████████████████████████████████| 10/10 [00:06<00:00,  6.23s/trial, best loss: 0.3831884579111654]
Beginning fold 5 of 10
100%|███████████████████████████████████████████████| 10/10 [00:06<00:00,  6.51s/trial, best loss: 0.36617633698744434]
Beginning fold 6 of 10
100%|████████████████████████████████████████████████| 10/10 [00:04<00:00,  5.00s/trial, best loss: 0.3667079139361499]
Beginning fold 7 of 10
100%|██████████████████████████████████████████████████| 6/6 [00:06<00:00,  6.30s/trial, best loss: 0.3711626920706437]

100%|████████████████████████████████████████████████| 10/10 [00:06<00:00,  6.10s/trial, best loss: 0.3711626920706437]
Beginning fold 8 of 10
100%|███████████████████████████████████████████████| 10/10 [00:17<00:00, 17.31s/trial, best loss: 0.36377313918012755]
Beginning fold 9 of 10
100%|███████████████████████████████████████████████| 10/10 [00:06<00:00,  6.55s/trial, best loss: 0.36299657783515515]
Beginning fold 10 of 10
100%|███████████████████████████████████████████████| 10/10 [00:06<00:00,  6.23s/trial, best loss: 0.35929637976900397]
Retraining model on full dataset
100%|████████████████████████████████████████████████| 10/10 [00:07<00:00,  7.90s/trial, best loss: 0.3619515646693182]


In [10]:
# CLASSIFIER kNN with bandwidth chuncks
unique_id = 'kNN_'+ d1 +'_full' # Unique ID for output file paths
clf = k_neighbors_classifier(unique_id)  # Classifier
AZKABANML.main_classify(df_sim_full, clf, unique_id, classifypath, preprocessing=[normalizer(unique_id, norm='l2')])

Beginning fold 1 of 10
100%|███████████████████████████████████████████████| 10/10 [00:10<00:00, 10.13s/trial, best loss: 0.08222305881135783]
Beginning fold 2 of 10
100%|█████████████████████████████████████████████████| 10/10 [00:07<00:00,  7.42s/trial, best loss: 0.077175126467948]
Beginning fold 3 of 10
100%|█████████████████████████████████████████████████| 10/10 [00:09<00:00,  9.56s/trial, best loss: 0.085437743543758]
Beginning fold 4 of 10
100%|███████████████████████████████████████████████| 10/10 [00:10<00:00, 10.33s/trial, best loss: 0.08104648838261763]
Beginning fold 5 of 10
100%|███████████████████████████████████████████████| 10/10 [00:09<00:00,  9.75s/trial, best loss: 0.09107284214150013]
Beginning fold 6 of 10
100%|███████████████████████████████████████████████| 10/10 [00:07<00:00,  7.45s/trial, best loss: 0.08580805640545719]
Beginning fold 7 of 10
100%|█████████████████████████████████████████████████| 6/6 [00:10<00:00, 10.94s/trial, best loss: 0.10533545882816486]

100%|█████████████████████████████████████████████████| 10/10 [00:10<00:00, 10.98s/trial, best loss: 0.084174846230479]
Beginning fold 8 of 10
100%|████████████████████████████████████████████████| 10/10 [00:08<00:00,  8.32s/trial, best loss: 0.0965753893061777]
Beginning fold 9 of 10
100%|███████████████████████████████████████████████| 10/10 [00:08<00:00,  9.00s/trial, best loss: 0.08046007242378517]
Beginning fold 10 of 10
100%|████████████████████████████████████████████████| 10/10 [00:10<00:00, 10.74s/trial, best loss: 0.0833364265074239]
Retraining model on full dataset
100%|███████████████████████████████████████████████| 10/10 [00:09<00:00,  9.02s/trial, best loss: 0.07853219251198174]


## Linear
Balanced + PP